In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt  
from linearmodels import PanelOLS
import statsmodels.api as sm

from econtools import read, outreg, table_statrow, write_notes
import econtools as econ
import econtools.metrics as mt

#!pip install pylatex
#!pip install pylatex[matrices]
from econtools import read, outreg, table_statrow, write_notes
from pylatex import Document, Section, Subsection, Tabular, Math, TikZ, Axis, \
    Plot, Figure, Matrix, Alignat
from pylatex.utils import italic
import os

df = pd.read_stata('data/Authority.dta')

#construct work category dummy
df['OG03_dummy'] = 0
df.loc[(df['work_category']=='OG03')&(df['work_category']!=''),'OG03_dummy'] = 1

df['OG01_dummy'] = 0
df.loc[(df['work_category']=='OG01')&(df['work_category']!=''),'OG01_dummy'] = 1

df['OG_rest_dummy'] = 0
df.loc[(df['OG01_dummy']!=1)&(df['OG03_dummy']!=1)&(df['work_category']!=''),'OG_rest_dummy'] = 1

df['OG_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OG','OG_dummy'] = 1

df['OS_dummy'] = 0
df.loc[df['work_category'].str[0:2] == 'OS','OS_dummy'] = 1

#treated vs controls
df['trend'] = df['year'] - 1999

df['trend_treat'] = df['trend']
df.loc[(df['authority_code']!=3090272)&(df['authority_code']!=3070001),'trend_treat'] = 0
#15225 real change made / no zeros in df['trend']
#print(df['trend_treat'].value_counts()) chekced

df['trend_control'] = df['trend']
df.loc[(df['authority_code']==3090272)|(df['authority_code']==3070001),'trend_control'] = 0
#902 real change made
#print(df['trend_control'].value_counts()) checked

#PA specifics
df = df.sort_values(by='authority_code',ascending=True)
#auth = econ.group_id(df, cols = 'authority_code')
#print(auth) #dataframe

auth_list = df['authority_code'].values.tolist()
auth_list = list(set(auth_list))
#겹치는 부분 = authority_code야
#id_auth = group_id + 1 if df의 auth code == auth의 code

df['id_auth'] = 0
for i in range(len(df)):
    for j in range(len(auth_list)):
        if df.loc[i,'authority_code'] == auth_list[j]:
            df.loc[i,'id_auth'] = j+1

work_dum = pd.get_dummies(df['work_category'])
year_dum = pd.get_dummies(df['year'])
work_list = list(work_dum.columns)
year_list = list(year_dum.columns)

df_dum = pd.concat([year_dum, work_dum],axis = 1)
df = pd.concat([df, df_dum],axis = 1)

# test: switch to sm.ols

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

def areg(formula,data=None,absorb=None,cluster=None): 

    y,X = patsy.dmatrices(formula,data,return_type='dataframe')

    ybar = y.mean()
    y = y -  y.groupby(data[absorb]).transform('mean') + ybar

    Xbar = X.mean()
    X = X - X.groupby(data[absorb]).transform('mean') + Xbar

    reg = sm.OLS(y,X)
    # Account for df loss from FE transform
    reg.df_resid -= (data[absorb].nunique() - 1)

    return reg.fit(cov_type='cluster',cov_kwds={'groups':data[cluster].values})

def areg('discount~fpsb_auction+reserve_price+municipality')

In [15]:
#switch to sm.ols :table2 (1,2)
#id_auth 삭제, reg_col에서 muni 삭제하는 거 삭제(직접삭제만 남겨둠), 'o'=nan 삭제하는 거 삭제(품고간다)
#이렇게했을 때 잘 돌아가는지 보고, sm.ols돌리기
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']
year = pd.Categorical(df_reg_co.year)
df_reg_co = df_reg_co.set_index(['year','authority_code'])
df_reg_co['year'] = year
# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
#        idx = df_reg_co[df_reg_co[o].isnull()==True].index
#        df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_reg_co.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)

    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
    
    exog = exog_var + reg_col
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    mod = sm.OLS(df_reg_co[o],df_reg_co[exog]).fit(cov_type='cluster',cov_kwds={"groups":df_reg_co['auth_anno']})
    res = mod.summary()

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\base\model.py:1832: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 31
  'rank is %d' % (J, J_), ValueWarning)


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal

In [39]:
#f.write r2 and N
#talbe 2 col2 fianl version
#to make iterable
#스엉공..
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #idx = df_reg_co[df_reg_co[o].isnull()==True].index
    #df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
        #exog.remove('municipality')

    if o == 'discount':
        fe_reg_discount = mt.reg(df_reg_co, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    elif o == 'delay_ratio':
        fe_reg_delay = mt.reg(df_reg_co, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    elif o == 'overrun_ratio':
        fe_reg_overrun = mt.reg(df_reg_co, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    else :
        fe_reg_award = mt.reg(df_reg_co, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
        

result_path = 'my_result.tex'
with open(result_path,'w') as f:
    reg = (fe_reg_discount, fe_reg_delay, fe_reg_overrun, fe_reg_award )
    table_string = outreg(reg, ['fpsb_auction'], ['First Price Auction'], digits = 3)
    table_string += '\\\ \n'
    table_string += table_statrow("R$^2$", [x.r2 for x in reg], digits =3)
    table_string += table_statrow("Number of Observation", [x.N for x in reg])
        #table_string += table_statrwo("Stdard Error",[iterable])
    f.write(table_string + '\n')

KeyError: 'turin_co_sample'

In [38]:
# 완성된 my result 읽어보기
df_string = pd.DataFrame({'table2':[table_string]})
#df['strip'] = df['table2'].str.strip()
#df['lstrip'] = df['table2'].str.lstrip()
#df['rstrip'] = df['table2'].str.rstrip()
df[['email_partition_1', 'email_partition_2', 'email_partition_3']] = df['table2'].str.split('&', n=3, expand=True)
# = df['table2'].str.partition(sep='&')
df


ValueError: Columns must be same length as key

# Cal2 = multicolinearity check and then delete NaN or vif > 10

In [47]:
#talbe 2 col2 fianl version
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
result_path = 'my_result.tex'
with open(result_path,'w') as f:
    for o in outcome:
    #days_to_award(4,6)
        idx = df_reg_co[df_reg_co[o].isnull()==True].index
        df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
        reg_col = []
        for i in work_list:
            reg_col.append(i)
        for j in year_list:
            reg_col.append(j)
        exog_var = ['fpsb_auction','id_auth','reserve_price','municipality']
        exog = exog_var + reg_col 


    #check multicollinearity
        X = df_name.loc[:,exog]
        vif = calc_vif(X)
    #print(vif)


    #delete from col list
        for i in range(len(vif)):
            if np.isnan(vif.loc[i, 'VIF']) == True:
                reg_col.remove(vif.loc[i, 'variables'])
            elif vif.loc[i,'VIF'] > 10:
                for j in exog_var:
                    if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                        exog_var.remove(vif.loc[i,'variables'])
                
        exog = exog_var + reg_col
        exog.remove('id_auth')
        exog.remove(2000)
        exog.remove('OG01')
    #exog.remove('municipality')

    #print(exog) #check
        fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    
    #plot
        reg = (fe_reg)
        table_string = outreg(reg, ['fpsb_auction','R-squared','N'], ['First Price Auction','R$^2$', 'Number of Observations'], digits = 3)
        table_string += '\\\ \n'
        #table_string += table_statrow("R$^2$", [exog.r2 for exog in reg], digits =3)
        #table_string += table_statrwo("Stdard Error",[iterable])
        f.write(table_string + '\n')

# Col 4: ctrl_pop& iwork

In [12]:
#talbe 2 col4 fianl version
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

Dependent variable:	discount
N:			1275
R-squared:		0.6140
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  145
Fixed effects by:	authority_code
  No. of FE:		  24
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  12.023 1.469  8.187 0.000  9.120  14.926
reserve_price  0.000 0.000  5.221 0.000  0.000   0.000
OG02          -0.045 0.694 -0.065 0.948 -1.416   1.326
OG03          -0.696 0.582 -1.197 0.233 -1.845   0.454
OG04          -0.326 0.557 -0.585 0.560 -1.426   0.775
OG06          -3.037 1.465 -2.073 0.040 -5.933  -0.141
OG07           0.948 2.510  0.378 0.706 -4.014   5.910
OG08           1.041 1.632  0.638 0.525 -2.186   4.267
OG09           2.600 1.163  2.236 0.027  0.301   4.898
OG10           8.371 0.551 15.183 0.000  7.282   9.461
OG11           5.921 1.061  5.582 0.000  3.824   8.017
OG12           6.018 2.011  2.993 0.003  2.043   9.992
OG13          -2.649 1.176 -2.252 0.026 -4.973  -0.324
OS01          -1.32

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	delay_ratio
N:			1084
R-squared:		0.1483
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  140
Fixed effects by:	authority_code
  No. of FE:		  24
                coeff     se      t   p>t   CI_low CI_high
fpsb_auction   27.977 10.144  2.758 0.007    7.919  48.034
reserve_price  -0.000  0.000 -2.195 0.030   -0.000  -0.000
OG02           -0.897  9.372 -0.096 0.924  -19.427  17.634
OG03            8.247  6.089  1.354 0.178   -3.793  20.287
OG04          -23.524  9.064 -2.595 0.010  -41.445  -5.602
OG06           73.854 27.002  2.735 0.007   20.466 127.242
OG07           -9.433 23.017 -0.410 0.683  -54.942  36.076
OG08            9.899 14.982  0.661 0.510  -19.723  39.522
OG10           55.334  8.217  6.734 0.000   39.087  71.582
OG11           -0.780 12.131 -0.064 0.949  -24.765  23.205
OG12           41.140 51.854  0.793 0.429  -61.385 143.664
OG13           59.584  7.360  8.095 0.000   45.031  74.137
OS01           -5.5

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			1049
R-squared:		0.2139
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  139
Fixed effects by:	authority_code
  No. of FE:		  24
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    4.325 2.134  2.027 0.045   0.106   8.545
reserve_price   0.000 0.000  2.584 0.011   0.000   0.000
OG02           -3.161 1.635 -1.933 0.055  -6.395   0.073
OG03           -2.380 1.227 -1.940 0.054  -4.805   0.046
OG06           -3.003 2.920 -1.029 0.306  -8.777   2.770
OG07           -2.162 5.244 -0.412 0.681 -12.531   8.208
OG08           -4.946 2.361 -2.094 0.038  -9.615  -0.277
OG09          -10.201 3.852 -2.648 0.009 -17.818  -2.584
OG10           -5.357 2.687 -1.993 0.048 -10.671  -0.043
OG11           -2.780 2.084 -1.334 0.184  -6.901   1.341
OG12           -2.512 4.353 -0.577 0.565 -11.120   6.095
OG13           -1.638 3.902 -0.420 0.675  -9.353   6.077
OS01           -7.789 2.293 -3.397 0.001 -1

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	days_to_award
N:			777
R-squared:		0.5472
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  122
Fixed effects by:	authority_code
  No. of FE:		  24
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   23.539 11.643  2.022 0.045   0.488  46.590
reserve_price   0.000  0.000  1.626 0.106  -0.000   0.000
OG02            2.289  6.457  0.354 0.724 -10.495  15.072
OG03            9.851  4.839  2.036 0.044   0.271  19.430
OG06          -10.901 37.390 -0.292 0.771 -84.926  63.123
OG07          -29.005  7.956 -3.646 0.000 -44.757 -13.254
OG08           13.752 13.089  1.051 0.295 -12.160  39.664
OG09           -9.201 11.743 -0.784 0.435 -32.449  14.047
OG10          -23.677  9.620 -2.461 0.015 -42.723  -4.632
OG11           17.610 13.795  1.276 0.204  -9.702  44.921
OG12           -5.236  7.016 -0.746 0.457 -19.126   8.654
OG13          -18.139 11.913 -1.523 0.130 -41.724   5.447
OS01           -8.239 11.043 -0

# Col 6: ctrl_pop_exp

In [13]:
#talbe 2 col6 fianl version
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for i in work_list:
        reg_col.append(i)
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction','reserve_price','municipality']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
        elif vif.loc[i,'VIF'] > 10:
            for j in exog_var:
                if str(vif.loc[i,'variables']) is j and vif.loc[i,'variables'] is not 'fpsb_auction' and vif.loc[i,'variables'] is not 'id_auth':
                    exog_var.remove(vif.loc[i,'variables'])
                
    exog = exog_var + reg_col
    #exog.remove('id_auth')
    exog.remove(2000)
    exog.remove('OG01')
    exog.remove('municipality')

    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			880
R-squared:		0.6444
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  73
Fixed effects by:	authority_code
  No. of FE:		  11
               coeff    se      t   p>t CI_low CI_high
fpsb_auction  12.258 1.380  8.880 0.000  9.506  15.009
reserve_price  0.000 0.000  4.255 0.000  0.000   0.000
OG02          -0.958 0.791 -1.212 0.230 -2.535   0.618
OG03          -0.561 0.745 -0.753 0.454 -2.047   0.924
OG04          -0.382 0.725 -0.527 0.600 -1.828   1.063
OG06          -1.935 1.355 -1.429 0.157 -4.636   0.765
OG07           4.209 5.161  0.816 0.417 -6.079  14.497
OG08           0.524 1.660  0.316 0.753 -2.786   3.834
OG10           8.428 0.618 13.632 0.000  7.195   9.660
OG11           5.903 1.475  4.003 0.000  2.963   8.843
OG12           6.723 2.211  3.040 0.003  2.315  11.131
OS01          -2.257 1.326 -1.702 0.093 -4.899   0.386
OS02           9.543 1.465  6.512 0.000  6.622  12.464
OS05           5.700 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	delay_ratio
N:			747
R-squared:		0.1472
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  71
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff     se      t   p>t  CI_low CI_high
fpsb_auction   39.285 12.001  3.274 0.002  15.350  63.219
reserve_price  -0.000  0.000 -2.248 0.028  -0.000  -0.000
OG02           -7.784 11.446 -0.680 0.499 -30.612  15.044
OG03            4.890  7.126  0.686 0.495  -9.324  19.103
OG04          -16.448 10.805 -1.522 0.132 -37.998   5.101
OG06          109.160 19.826  5.506 0.000  69.618 148.701
OG07           47.214 26.145  1.806 0.075  -4.931  99.359
OG08            8.643 15.792  0.547 0.586 -22.854  40.140
OG10           54.134 12.030  4.500 0.000  30.140  78.128
OG11            8.686 14.587  0.595 0.553 -20.406  37.778
OG12           38.608 63.160  0.611 0.543 -87.360 164.577
OS01           -7.694  7.427 -1.036 0.304 -22.508   7.119
OS02            7.514  9.203  0.81

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			742
R-squared:		0.2214
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  70
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff    se      t   p>t  CI_low CI_high
fpsb_auction    5.713 2.196  2.601 0.011   1.332  10.095
reserve_price   0.000 0.000  3.205 0.002   0.000   0.000
OG02           -3.154 1.674 -1.885 0.064  -6.494   0.185
OG03           -3.991 1.493 -2.674 0.009  -6.969  -1.013
OG06           -1.192 4.186 -0.285 0.777  -9.543   7.160
OG07            6.924 2.616  2.647 0.010   1.706  12.142
OG08           -5.954 2.435 -2.445 0.017 -10.812  -1.096
OG10           -6.117 2.938 -2.082 0.041 -11.979  -0.255
OG11           -2.075 1.891 -1.097 0.276  -5.848   1.698
OG12           -3.836 4.632 -0.828 0.410 -13.076   5.404
OS01           -9.908 2.018 -4.909 0.000 -13.935  -5.882
OS02            5.709 5.305  1.076 0.286  -4.874  16.292
OS05          -14.304 2.131 -6.711 0.000 -18.

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\stats\outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


Dependent variable:	days_to_award
N:			549
R-squared:		0.5654
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  64
Fixed effects by:	authority_code
  No. of FE:		  11
                coeff     se      t   p>t   CI_low CI_high
fpsb_auction   33.037 14.404  2.294 0.025    4.253  61.820
reserve_price   0.000  0.000  0.070 0.944   -0.000   0.000
OG02            0.370  8.248  0.045 0.964  -16.111  16.851
OG03            8.363  6.371  1.313 0.194   -4.368  21.094
OG06          -37.554 36.397 -1.032 0.306 -110.288  35.180
OG07          -20.655 10.336 -1.998 0.050  -41.311   0.001
OG08            9.494 14.354  0.661 0.511  -19.191  38.179
OG10          -21.701  9.066 -2.394 0.020  -39.818  -3.585
OG11            9.152 17.314  0.529 0.599  -25.448  43.751
OG12           -4.319  7.346 -0.588 0.559  -19.000  10.362
OS01          -18.338 11.046 -1.660 0.102  -40.411   3.736
OS02           24.825 14.051  1.767 0.082   -3.253  52.903
OS06           10.9

# Odd Cols

In [16]:
#talbe 2 odds fianl version
#col1
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1262
R-squared:		0.5047
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  101
Fixed effects by:	authority_code
  No. of FE:		  15
              coeff    se      t   p>t CI_low CI_high
fpsb_auction 13.095 1.609  8.137 0.000  9.902  16.288
2001.0        2.181 1.144  1.906 0.060 -0.089   4.450
2002.0        0.583 1.140  0.512 0.610 -1.678   2.844
2003.0       -0.243 1.111 -0.219 0.827 -2.446   1.961
2004.0        1.580 1.034  1.528 0.130 -0.472   3.631
2005.0        3.764 1.155  3.259 0.002  1.473   6.056
2006.0        4.024 1.281  3.140 0.002  1.482   6.566

Dependent variable:	delay_ratio
N:			1110
R-squared:		0.0853
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  99
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  25.232 12.051  2.094 0.039   1.317  49.147
2001.0       -19.404  4.699 -

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			1092
R-squared:		0.1113
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  98
Fixed effects by:	authority_code
  No. of FE:		  15
              coeff    se      t   p>t CI_low CI_high
fpsb_auction  7.201 1.986  3.626 0.000  3.259  11.144
2001.0       -1.065 1.433 -0.744 0.459 -3.908   1.778
2002.0       -0.992 1.438 -0.690 0.492 -3.846   1.862
2003.0        1.345 2.028  0.663 0.509 -2.680   5.370
2004.0       -0.993 1.468 -0.677 0.500 -3.907   1.920
2005.0        0.637 1.388  0.459 0.647 -2.119   3.392
2006.0        1.836 1.678  1.094 0.277 -1.494   5.165

Dependent variable:	days_to_award
N:			777
R-squared:		0.5539
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  88
Fixed effects by:	authority_code
  No. of FE:		  15
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  28.700 10.779  2.663 0.009   7.275  50.125
2001.0         1.377  7.

In [17]:
#talbe 2 odds fianl version
#col3
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_pop_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

#value chekced

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			1275
R-squared:		0.4927
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  145
Fixed effects by:	authority_code
  No. of FE:		  24
              coeff    se      t   p>t CI_low CI_high
fpsb_auction 13.323 1.770  7.527 0.000  9.824  16.821
2001.0        1.604 1.216  1.319 0.189 -0.799   4.006
2002.0       -0.332 1.224 -0.271 0.787 -2.752   2.088
2003.0       -0.905 1.161 -0.780 0.437 -3.201   1.390
2004.0        1.084 1.081  1.003 0.318 -1.053   3.221
2005.0        2.992 1.223  2.447 0.016  0.576   5.409
2006.0        3.426 1.391  2.462 0.015  0.676   6.176

Dependent variable:	delay_ratio
N:			1084
R-squared:		0.1109
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  140
Fixed effects by:	authority_code
  No. of FE:		  24
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  19.361  9.814  1.973 0.050  -0.042  38.764
2001.0       -14.853  7.588 

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			1049
R-squared:		0.1849
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  139
Fixed effects by:	authority_code
  No. of FE:		  24
              coeff    se      t   p>t CI_low CI_high
fpsb_auction  5.136 2.157  2.381 0.019  0.871   9.400
2001.0       -0.888 1.607 -0.552 0.582 -4.066   2.290
2002.0       -0.154 1.516 -0.102 0.919 -3.151   2.842
2003.0        2.955 2.168  1.363 0.175 -1.331   7.241
2004.0        0.363 1.687  0.215 0.830 -2.973   3.699
2005.0        4.072 1.951  2.087 0.039  0.215   7.929
2006.0        4.457 1.975  2.257 0.026  0.552   8.361

Dependent variable:	days_to_award
N:			777
R-squared:		0.5310
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  122
Fixed effects by:	authority_code
  No. of FE:		  24
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  27.286 10.503  2.598 0.011   6.493  48.078
2001.0         5.200  

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


In [20]:
#talbe 2 odds fianl version
#col5
df_reg_co = df[(df['turin_co_sample']==1)&(df['ctrl_pop_exp_turin_co_sample']==1)&(df['post_experience']>= 5) & (df['pre_experience']>=5) &(df['post_experience'].isnull() == False ) & (df['pre_experience'].isnull()==False)&(df['missing']==0)]
outcome = ['discount', 'delay_ratio', 'overrun_ratio', 'days_to_award']

# mutlicollinearity function
from statsmodels.stats.outliers_influence import variance_inflation_factor
def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X.columns
    vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

    return(vif)

#iteration
for o in outcome:
    #days_to_award(4,6)
    idx = df_reg_co[df_reg_co[o].isnull()==True].index
    df_name = df_reg_co.drop(idx)
    
    #vif cal
    #first, make a column list
    reg_col = []
    for j in year_list:
        reg_col.append(j)
    exog_var = ['fpsb_auction']
    exog = exog_var + reg_col 


    #check multicollinearity
    X = df_name.loc[:,exog]
    vif = calc_vif(X)
    #print(vif)


    #delete from col list
    for i in range(len(vif)):
        if np.isnan(vif.loc[i, 'VIF']) == True:
            reg_col.remove(vif.loc[i, 'variables'])
                
    exog = exog_var + reg_col
    exog.remove(2000)


    #print(exog) #check
    fe_reg = mt.reg(df_name, o, exog, fe_name = 'authority_code', cluster = 'auth_anno')
    print(fe_reg)

#value chekced

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss
C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	discount
N:			880
R-squared:		0.5259
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  73
Fixed effects by:	authority_code
  No. of FE:		  11
              coeff    se      t   p>t CI_low CI_high
fpsb_auction 13.712 1.720  7.974 0.000 10.284  17.140
2001.0        3.128 1.412  2.215 0.030  0.313   5.943
2002.0        0.419 1.633  0.256 0.799 -2.837   3.674
2003.0       -1.166 1.608 -0.725 0.471 -4.372   2.040
2004.0        1.491 1.462  1.020 0.311 -1.423   4.405
2005.0        3.444 1.624  2.120 0.037  0.206   6.682
2006.0        4.029 1.748  2.305 0.024  0.544   7.513

Dependent variable:	delay_ratio
N:			747
R-squared:		0.0951
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  71
Fixed effects by:	authority_code
  No. of FE:		  11
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  27.729 10.784  2.571 0.012   6.222  49.236
2001.0       -19.435  5.029 -3.8

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


Dependent variable:	overrun_ratio
N:			742
R-squared:		0.1601
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  70
Fixed effects by:	authority_code
  No. of FE:		  11
              coeff    se      t   p>t CI_low CI_high
fpsb_auction  7.249 2.234  3.244 0.002  2.792  11.706
2001.0       -0.323 1.805 -0.179 0.859 -3.925   3.279
2002.0        0.232 1.807  0.128 0.898 -3.372   3.836
2003.0        1.893 2.985  0.634 0.528 -4.063   7.848
2004.0       -1.212 1.967 -0.616 0.540 -5.137   2.712
2005.0        1.652 1.927  0.857 0.394 -2.192   5.496
2006.0        2.696 2.196  1.228 0.224 -1.685   7.077

Dependent variable:	days_to_award
N:			549
R-squared:		0.5465
Estimation method:	OLS
VCE method:		Cluster
  Cluster variable:	  auth_anno
  No. of clusters:	  64
Fixed effects by:	authority_code
  No. of FE:		  11
               coeff     se      t   p>t  CI_low CI_high
fpsb_auction  34.053 11.717  2.906 0.005  10.639  57.467
2001.0         9.098  8.8

C:\Users\thxkn\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1685: RuntimeWarning: invalid value encountered in true_divide
  return 1 - self.ssr/self.centered_tss


In [26]:
print(fe_reg['coeff'])

TypeError: 'Results' object is not subscriptable